In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

from keras.models import Sequential
from keras.layers import Dense
from keras import metrics

In [2]:
def get_confusion_matrix(true_pos, false_pos, true_neg, false_neg):
    conf_matrix = np.array([
                            [true_pos, false_pos],
                            [false_neg, true_neg]
                           ])
    
    return pd.DataFrame(conf_matrix, columns=['1', '0'], index=['1', '0'])

In [3]:
df = pd.read_csv('jazz.csv', sep='|')

In [4]:
# Define features to analyze
features = [
            'danceability',
            'energy',
            'speechiness',
            'acousticness',
            'instrumentalness',
            'liveness',
            'valence',
            'num_samples',
            'end_of_fade_in',
            'loudness',
            'tempo',
            'key',
            'mode',
            'bars_num',
            'bars_duration_var',
            'beats_duration_var',
            'sections_num',
            'sections_duration_mean',
            'sections_duration_var',
            'loudness_var',
            'tempo_var',
            'key_var',
            'mode_var',
            'segments_duration_var',
            'segments_duration_mean',
            'pitches_mean',
            'pitches_var',
            'timbre_mean',
            'timbre_var',
            'tatums_duration_var'
           ]

df = df[features+['label']]

In [5]:
df[df['label']==1].shape

(953, 31)

Split dataset into training, validation, and test. 60/20/20 gives ~570 positive training samples, and ~190 each of positive validation and test samples. Quite small, but hopefully big enough to be meaningful.

In [6]:
X = df[features].copy()
Y = df['label'].copy()

X_trainval, X_test, y_trainval, y_test = train_test_split(
                                                          X,
                                                          Y, 
                                                          stratify = Y,
                                                          test_size = 0.2,
                                                          random_state = 42
                                                         )
X_train, X_val, y_train, y_val = train_test_split(
                                                  X_trainval,
                                                  y_trainval,
                                                  stratify = y_trainval,
                                                  test_size = 0.25,
                                                  random_state = 42
                                                 )

# Start building model

In [7]:
# First fit how to scale data for the model
pipeline = Pipeline(steps=[('scaler', StandardScaler()), ('imputer', SimpleImputer(strategy='median'))])

scale_model = pipeline.fit(X_train)
X_train = scale_model.transform(X_train)
X_val = scale_model.transform(X_val)
X_test = scale_model.transform(X_test)

In [8]:
model = Sequential()
model.add(Dense(4, input_dim=len(features), activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

compile_metrics = [
                   metrics.Precision(),
                   metrics.TruePositives(),
                   metrics.TrueNegatives(),
                   metrics.FalsePositives(),
                   metrics.FalseNegatives(),
                   metrics.AUC(curve='PR')
                  ]
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=compile_metrics)

model.fit(X_train, y_train, epochs=250, batch_size=10)

Epoch 1/250
186/186 [==============================] - 0s 877us/step - loss: 0.6839 - precision: 0.3362 - true_positives: 195.0000 - true_negatives: 897.0000 - false_positives: 385.0000 - false_negatives: 377.0000 - auc: 0.3342
Epoch 2/250
186/186 [==============================] - 0s 703us/step - loss: 0.5926 - precision: 0.6000 - true_positives: 96.0000 - true_negatives: 1218.0000 - false_positives: 64.0000 - false_negatives: 476.0000 - auc: 0.4598
Epoch 3/250
186/186 [==============================] - 0s 660us/step - loss: 0.5459 - precision: 0.6547 - true_positives: 91.0000 - true_negatives: 1234.0000 - false_positives: 48.0000 - false_negatives: 481.0000 - auc: 0.5340
Epoch 4/250
186/186 [==============================] - 0s 677us/step - loss: 0.5112 - precision: 0.6702 - true_positives: 128.0000 - true_negatives: 1219.0000 - false_positives: 63.0000 - false_negatives: 444.0000 - auc: 0.5835
Epoch 5/250
186/186 [==============================] - 0s 700us/step - loss: 0.4845 - prec

186/186 [==============================] - 0s 721us/step - loss: 0.3789 - precision: 0.7366 - true_positives: 400.0000 - true_negatives: 1139.0000 - false_positives: 143.0000 - false_negatives: 172.0000 - auc: 0.7771
Epoch 37/250
186/186 [==============================] - 0s 688us/step - loss: 0.3782 - precision: 0.7256 - true_positives: 402.0000 - true_negatives: 1130.0000 - false_positives: 152.0000 - false_negatives: 170.0000 - auc: 0.7763
Epoch 38/250
186/186 [==============================] - 0s 723us/step - loss: 0.3774 - precision: 0.7315 - true_positives: 406.0000 - true_negatives: 1133.0000 - false_positives: 149.0000 - false_negatives: 166.0000 - auc: 0.7807
Epoch 39/250
186/186 [==============================] - 0s 685us/step - loss: 0.3766 - precision: 0.7341 - true_positives: 403.0000 - true_negatives: 1136.0000 - false_positives: 146.0000 - false_negatives: 169.0000 - auc: 0.7813
Epoch 40/250
186/186 [==============================] - 0s 685us/step - loss: 0.3762 - precis

186/186 [==============================] - 0s 696us/step - loss: 0.3582 - precision: 0.7638 - true_positives: 414.0000 - true_negatives: 1154.0000 - false_positives: 128.0000 - false_negatives: 158.0000 - auc: 0.8032
Epoch 72/250
186/186 [==============================] - 0s 704us/step - loss: 0.3585 - precision: 0.7596 - true_positives: 414.0000 - true_negatives: 1151.0000 - false_positives: 131.0000 - false_negatives: 158.0000 - auc: 0.8008
Epoch 73/250
186/186 [==============================] - 0s 716us/step - loss: 0.3574 - precision: 0.7595 - true_positives: 420.0000 - true_negatives: 1149.0000 - false_positives: 133.0000 - false_negatives: 152.0000 - auc: 0.8055
Epoch 74/250
186/186 [==============================] - 0s 688us/step - loss: 0.3576 - precision: 0.7646 - true_positives: 419.0000 - true_negatives: 1153.0000 - false_positives: 129.0000 - false_negatives: 153.0000 - auc: 0.8020
Epoch 75/250
186/186 [==============================] - 0s 720us/step - loss: 0.3581 - precis

186/186 [==============================] - 0s 730us/step - loss: 0.3512 - precision: 0.7711 - true_positives: 421.0000 - true_negatives: 1157.0000 - false_positives: 125.0000 - false_negatives: 151.0000 - auc: 0.8092
Epoch 107/250
186/186 [==============================] - 0s 707us/step - loss: 0.3511 - precision: 0.7717 - true_positives: 426.0000 - true_negatives: 1156.0000 - false_positives: 126.0000 - false_negatives: 146.0000 - auc: 0.8099
Epoch 108/250
186/186 [==============================] - 0s 716us/step - loss: 0.3517 - precision: 0.7719 - true_positives: 423.0000 - true_negatives: 1157.0000 - false_positives: 125.0000 - false_negatives: 149.0000 - auc: 0.8106
Epoch 109/250
186/186 [==============================] - 0s 755us/step - loss: 0.3509 - precision: 0.7715 - true_positives: 422.0000 - true_negatives: 1157.0000 - false_positives: 125.0000 - false_negatives: 150.0000 - auc: 0.8103
Epoch 110/250
186/186 [==============================] - 0s 709us/step - loss: 0.3504 - pr

186/186 [==============================] - 0s 767us/step - loss: 0.3476 - precision: 0.7750 - true_positives: 427.0000 - true_negatives: 1158.0000 - false_positives: 124.0000 - false_negatives: 145.0000 - auc: 0.8140
Epoch 141/250
186/186 [==============================] - 0s 914us/step - loss: 0.3471 - precision: 0.7709 - true_positives: 424.0000 - true_negatives: 1156.0000 - false_positives: 126.0000 - false_negatives: 148.0000 - auc: 0.8155
Epoch 142/250
186/186 [==============================] - 0s 757us/step - loss: 0.3471 - precision: 0.7730 - true_positives: 429.0000 - true_negatives: 1156.0000 - false_positives: 126.0000 - false_negatives: 143.0000 - auc: 0.8147
Epoch 143/250
186/186 [==============================] - 0s 940us/step - loss: 0.3469 - precision: 0.7722 - true_positives: 427.0000 - true_negatives: 1156.0000 - false_positives: 126.0000 - false_negatives: 145.0000 - auc: 0.8155
Epoch 144/250
186/186 [==============================] - 0s 773us/step - loss: 0.3469 - pr

186/186 [==============================] - 0s 809us/step - loss: 0.3438 - precision: 0.7695 - true_positives: 424.0000 - true_negatives: 1155.0000 - false_positives: 127.0000 - false_negatives: 148.0000 - auc: 0.8194   
Epoch 175/250
186/186 [==============================] - 0s 756us/step - loss: 0.3431 - precision: 0.7845 - true_positives: 426.0000 - true_negatives: 1165.0000 - false_positives: 117.0000 - false_negatives: 146.0000 - auc: 0.8208
Epoch 176/250
186/186 [==============================] - 0s 775us/step - loss: 0.3434 - precision: 0.7770 - true_positives: 425.0000 - true_negatives: 1160.0000 - false_positives: 122.0000 - false_negatives: 147.0000 - auc: 0.8190
Epoch 177/250
186/186 [==============================] - 0s 771us/step - loss: 0.3438 - precision: 0.7823 - true_positives: 424.0000 - true_negatives: 1164.0000 - false_positives: 118.0000 - false_negatives: 148.0000 - auc: 0.8193
Epoch 178/250
186/186 [==============================] - 0s 777us/step - loss: 0.3437 -

186/186 [==============================] - 0s 781us/step - loss: 0.3409 - precision: 0.7706 - true_positives: 420.0000 - true_negatives: 1157.0000 - false_positives: 125.0000 - false_negatives: 152.0000 - auc: 0.8226
Epoch 210/250
186/186 [==============================] - 0s 790us/step - loss: 0.3408 - precision: 0.7804 - true_positives: 423.0000 - true_negatives: 1163.0000 - false_positives: 119.0000 - false_negatives: 149.0000 - auc: 0.8225
Epoch 211/250
186/186 [==============================] - 0s 753us/step - loss: 0.3418 - precision: 0.7673 - true_positives: 422.0000 - true_negatives: 1154.0000 - false_positives: 128.0000 - false_negatives: 150.0000 - auc: 0.8226
Epoch 212/250
186/186 [==============================] - 0s 769us/step - loss: 0.3403 - precision: 0.7733 - true_positives: 423.0000 - true_negatives: 1158.0000 - false_positives: 124.0000 - false_negatives: 149.0000 - auc: 0.8225
Epoch 213/250
186/186 [==============================] - 0s 781us/step - loss: 0.3407 - pr

186/186 [==============================] - 0s 805us/step - loss: 0.3391 - precision: 0.7681 - true_positives: 424.0000 - true_negatives: 1154.0000 - false_positives: 128.0000 - false_negatives: 148.0000 - auc: 0.8255
Epoch 244/250
186/186 [==============================] - 0s 805us/step - loss: 0.3385 - precision: 0.7617 - true_positives: 422.0000 - true_negatives: 1150.0000 - false_positives: 132.0000 - false_negatives: 150.0000 - auc: 0.8267
Epoch 245/250
186/186 [==============================] - 0s 803us/step - loss: 0.3382 - precision: 0.7698 - true_positives: 418.0000 - true_negatives: 1157.0000 - false_positives: 125.0000 - false_negatives: 154.0000 - auc: 0.8251
Epoch 246/250
186/186 [==============================] - 0s 816us/step - loss: 0.3379 - precision: 0.7681 - true_positives: 424.0000 - true_negatives: 1154.0000 - false_positives: 128.0000 - false_negatives: 148.0000 - auc: 0.8256
Epoch 247/250
186/186 [==============================] - 0s 826us/step - loss: 0.3391 - pr

# Evaluate model on validation set

In [9]:
loss_train, precision_train, true_positives_train, true_negatives_train, false_positives_train, false_negatives_train, auc_train = model.evaluate(X_train, y_train)

58/58 [==============================] - 0s 725us/step - loss: 0.3338 - precision: 0.7726 - true_positives: 428.0000 - true_negatives: 1156.0000 - false_positives: 126.0000 - false_negatives: 144.0000 - auc: 0.8323


In [10]:
loss_val, precision_val, true_positives_val, true_negatives_val, false_positives_val, false_negatives_val, auc_val = model.evaluate(X_val, y_val)

20/20 [==============================] - 0s 798us/step - loss: 0.4598 - precision: 0.6543 - true_positives: 123.0000 - true_negatives: 363.0000 - false_positives: 65.0000 - false_negatives: 67.0000 - auc: 0.7090


In [11]:
print('Loss for training set is {}, while loss for validation set is {}. This gives a difference of {}'\
      .format(round(loss_train,4), round(loss_val,4), round(loss_val-loss_train, 4)))

Loss for training set is 0.3338, while loss for validation set is 0.4598. This gives a difference of 0.126


In [12]:
print('AUC for training set is {}, while AUC for validation set is {}. This gives a difference of {}'\
      .format(round(auc_train, 2), round(auc_val, 2), round(auc_train-auc_val,2)))

AUC for training set is 0.83, while AUC for validation set is 0.71. This gives a difference of 0.12


In [13]:
val_conf_matrix = get_confusion_matrix(
                                       true_positives_val,
                                       false_positives_val,
                                       true_negatives_val,
                                       false_negatives_val
                                       )

print('Confusion_matrix:\n{}'.format(val_conf_matrix))

Confusion_matrix:
       1      0
1  123.0   65.0
0   67.0  363.0


In [14]:
print('True recall rate is {}'\
      .format(round(val_conf_matrix.loc['1', '1']/(val_conf_matrix.loc['1', '1']+val_conf_matrix.loc['0', '1']),2)))
print('Precision is {}'.format(precision_val))

True recall rate is 0.65
Precision is 0.6542553305625916


Looks very promising on validation set! Let's check test set

In [15]:
# loss_test, precision_test, true_positives_test, true_negatives_test, false_positives_test, false_negatives_test, auc_test = model.evaluate(X_test, y_test)

In [16]:
# print('AUC for test set is {}.'.format(round(auc_test,2)))

In [17]:
# confusion_matrix = get_confusion_matrix(
#                                         true_positives_test,
#                                         false_positives_test,
#                                         true_negatives_test,
#                                         false_negatives_test
#                                        )

In [18]:
# print(confusion_matrix)